## Download the data

In [ ]:
!git lfs install
!git clone https://huggingface.co/datasets/Hello-SimpleAI/HC3

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'HC3'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), 5.19 MiB | 5.32 MiB/s, done.
Filtering content: 100% (2/2), 123.19 MiB | 32.85 MiB/s, done.


## Imports

In [ ]:
!pip install jsonlines
import json
import jsonlines
from google.colab import drive
drive.mount('/content/gdrive')
COLAB_RUN = True
if COLAB_RUN:
  base_path = "./gdrive/MyDrive/ChatGPT-RetrievalQA/"
else:
  base_path = "./"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/gdrive


### normalizer for contents

In [ ]:
def normalize_content(content):
  return content.replace('\r', '').replace('\n', '').replace('\t', ' ')

##Dataset Creation

### Load all_jsonl from HC3 dataset

In [ ]:
empty_answers_ids = set([ "17497_C_1", "17509_C_2", "17513_C_1", "17640_C_2", "17751_C_2", "17784_C_1", "17874_C_1", "17906_C_2", "18061_C_1", "18061_C_2", "18113_C_1", "18113_C_2", "18162_C_1", "18162_C_2", "18253_C_1", "23832_C_0", "24020_C_0", "24283_C_1"])
all_jsonl = []
qrels_all_c = {}
qrels_all_h = {}
all_chatgpt_answerids = {}
all_human_answerids = {}
query_dict = {}
question_id = -1
chatgpt_answer_id_format = "{}_C_{}"#"{question_id}_C_{chatgpt_answer_id}"
human_answer_id_format = "{}_H_{}"#"{question_id}_H_{human_answer_id}"
with jsonlines.open('HC3/all.jsonl') as reader:
  for obj in reader:
    #assining id to question text
    question_id+=1
    obj["question_id"] = question_id
    query_dict[question_id]  = obj["question"]
    if question_id not in qrels_all_c:
      qrels_all_c[question_id] = []
      qrels_all_h[question_id] = []
    #assigning id to chatgpt answers
    obj["chatgpt_answers_ids"] = []
    chatgpt_answer_id = -1
    for chatgpt_answer in obj["chatgpt_answers"]:
      chatgpt_answer_id+=1
      chatgpt_answer_id_str = chatgpt_answer_id_format.format(question_id, chatgpt_answer_id)
      if chatgpt_answer_id_str in empty_answers_ids:
        del obj["chatgpt_answers"][chatgpt_answer_id]
      else:
        obj["chatgpt_answers_ids"].append(chatgpt_answer_id_str)
        qrels_all_c[question_id].append(chatgpt_answer_id_str)
        all_chatgpt_answerids[chatgpt_answer_id_str] = chatgpt_answer

    #assining id to human answers
    obj["human_answers_ids"] = []
    human_answer_id = -1
    for human_answer in obj["human_answers"]:
      human_answer_id+=1
      human_answer_id_str = human_answer_id_format.format(question_id, human_answer_id)
      obj["human_answers_ids"].append(human_answer_id_str)
      qrels_all_h[question_id].append(human_answer_id_str)
      all_human_answerids[human_answer_id_str] = human_answer
    # print(obj)
    all_jsonl.append(obj)

### Fixing an issue in the current version of HC3 dataset! 
There are 18 responses by ChatGPT that are empty in the current version of HC3 dataset. 
The ids of these answer after we assign ids to chatGPTresponses are: [17497_C_1, 17509_C_2, 17513_C_1, 17640_C_2, 17751_C_2, 17784_C_1, 17874_C_1, 17906_C_2, 18061_C_1, 18061_C_2, 18113_C_1, 18113_C_2, 18162_C_1, 18162_C_2, 18253_C_1, 23832_C_0, 24020_C_0, 24283_C_1]

For fixing this issue, we remove those responses that their id suffix is C_1 or C_2 (in the above cell we removed them) because it means that there is at least one response by chatGPT for them. However, two answers (**24020_C_0, 23832_C_0**) are the only response for the question and they are empty! So we give these question to ChatGPT and insert the answer of ChatGPT to these two answer manually!

In [ ]:
for empty_answer_id in empty_answers_ids:
  if empty_answer_id in all_chatgpt_answerids:
    print("deleting empty_answer_id: ", empty_answer_id)
    del all_chatgpt_answerids[empty_answer_id]

In [ ]:
print('question for 24020_C_0: ', query_dict[24020])
print('question for 23832_C_0: ', query_dict[23832])

question for 24020_C_0:  How to get a sure cure of acne ?hi, if you have been to the doctors before to get something checked out and they tell you that you have something similar to acne what can you ask for to get rid of them ?
question for 23832_C_0:  Kidney infection, weird twinges in bladder and uterus, lower back pain, fibroid in uterus. Bladder recovering from the infection?Hi, I have just had a kidney infection. No fever now and it seems to have cleared. However, I am feeling very weird twinges in my bladder or uterus . Not really a pain, although I do have lower back pain. Is it just the bladder recovering from the infection? I do have a fibroid in my uterus,I am five years into menopause.


In [ ]:
all_chatgpt_answerids['24020_C_0'] = "Acne is a common skin condition that affects many people, and there is no one-size-fits-all cure for it. However, there are several treatments that can help reduce its severity and improve the appearance of your skin. To get a sure cure for acne, you can consider the following steps: Consult a dermatologist: A dermatologist can diagnose your skin condition and determine the best treatment plan for you based on the type and severity of your acne. Use topical treatments: Topical creams and gels that contain active ingredients such as benzoyl peroxide, salicylic acid, and retinoids can help unclog pores and reduce inflammation. Take oral medications: If your acne is severe, your dermatologist may prescribe oral antibiotics to help reduce bacteria and inflammation. Consider light and laser therapies: Light and laser therapies, such as blue light therapy or photodynamic therapy, can help kill acne-causing bacteria and reduce inflammation. Maintain a healthy lifestyle: Eating a healthy diet, getting enough sleep, and reducing stress can also help improve your skin and prevent future breakouts. It's important to follow your dermatologist's recommendations and be patient, as some treatments may take several weeks or months to produce noticeable results."
all_chatgpt_answerids['23832_C_0'] = "It's always best to seek the advice of a doctor to get a proper diagnosis, but the symptoms you are describing could indicate a number of conditions. The twinges and lower back pain could be related to the fibroid in your uterus, which is a common condition. The fibroid could be pressing on the nerves in your back and causing pain. The bladder twinges could be related to the recent kidney infection, and your bladder could still be recovering from the infection. It's possible that the twinges could also be related to other factors such as bladder problems or even menopause-related changes. I would recommend visiting your doctor for an examination and to discuss your symptoms. Your doctor may want to run some tests to determine the cause of your symptoms and determine the best course of treatment."

In [ ]:
print('len all_jsonl[24020]["chatgpt_answers"]', len(all_jsonl[24020]["chatgpt_answers"]))
all_jsonl[24020]["chatgpt_answers"].append(all_chatgpt_answerids['24020_C_0'])
all_jsonl[24020]["chatgpt_answers_ids"].append('24020_C_0')
print('len all_jsonl[23832]["chatgpt_answers"]', len(all_jsonl[23832]["chatgpt_answers"]))
all_jsonl[23832]["chatgpt_answers"].append(all_chatgpt_answerids['23832_C_0'])
all_jsonl[23832]["chatgpt_answers_ids"].append('23832_C_0')

print('len all_jsonl[24020]["chatgpt_answers"]', len(all_jsonl[24020]["chatgpt_answers"]))
print('len all_jsonl[23832]["chatgpt_answers"]', len(all_jsonl[23832]["chatgpt_answers"]))

len all_jsonl[24020]["chatgpt_answers"] 0
len all_jsonl[23832]["chatgpt_answers"] 0
len all_jsonl[24020]["chatgpt_answers"] 1
len all_jsonl[23832]["chatgpt_answers"] 1


In [ ]:
# all_jsonl[20]

### Creating Collection-H (Colletion of responses that are produced by humans)

In [ ]:
#Collection-H
collection_h = []
output_path = base_path + "collection_h.tsv"
collection_line_format = "{id}\t{content}\n"
for obj in all_jsonl:
  for answer, answer_id in zip(obj["human_answers"], obj["human_answers_ids"]):
    collection_h.append(collection_line_format.format(id= answer_id, content= normalize_content(answer)))
f_w = open(output_path, "w+")
f_w.write("".join(collection_h))
f_w.close()

### Creating Collection-C (Colletion of responses that are produced by ChatGPT)

In [ ]:
#Collection-H
collection_c = []
output_path = base_path + "collection_c.tsv"
collection_line_format = "{id}\t{content}\n"
for obj in all_jsonl:
  for answer, answer_id in zip(obj["chatgpt_answers"], obj["chatgpt_answers_ids"]):
    collection_c.append(collection_line_format.format(id= answer_id, content= normalize_content(answer)))
f_w = open(output_path, "w+")
f_w.write("".join(collection_c))
f_w.close()

In [ ]:
!cat ./gdrive/MyDrive/ChatGPT-RetrievalQA/collection_c.tsv | grep 24020_C_0

24020_C_0	Acne is a common skin condition that affects many people, and there is no one-size-fits-all cure for it. However, there are several treatments that can help reduce its severity and improve the appearance of your skin. To get a sure cure for acne, you can consider the following steps: Consult a dermatologist: A dermatologist can diagnose your skin condition and determine the best treatment plan for you based on the type and severity of your acne. Use topical treatments: Topical creams and gels that contain active ingredients such as benzoyl peroxide, salicylic acid, and retinoids can help unclog pores and reduce inflammation. Take oral medications: If your acne is severe, your dermatologist may prescribe oral antibiotics to help reduce bacteria and inflammation. Consider light and laser therapies: Light and laser therapies, such as blue light therapy or photodynamic therapy, can help kill acne-causing bacteria and reduce inflammation. Maintain a healthy lifestyle: Eating a hea

### Queries

In [ ]:
#queries.tsv
queries = []
output_path = base_path + "queries.tsv"
queries_line_format = "{id}\t{content}\n"
for obj in all_jsonl:
  question = obj["question"]
  question_id = obj["question_id"]
  queries.append(queries_line_format.format(id= question_id, content= normalize_content(question)))
f_w = open(output_path, "w+")
f_w.write("".join(queries))
f_w.close()

### Split based on domain to train/valid/test (keep question ids as representation of train/valid/test set)

In [ ]:
from collections import Counter

all_questions = len(queries)
all_domains = list(set([obj["source"] for obj in all_jsonl]))
cnt_of_questions_per_domain = dict(Counter([obj["source"] for obj in all_jsonl]))
print("# of all_questions: ", all_questions)
print("all_domains: ", all_domains)
print("cnt_of_questions_per_domain: ", cnt_of_questions_per_domain)

# of all_questions:  24322
all_domains:  ['wiki_csai', 'medicine', 'reddit_eli5', 'finance', 'open_qa']
cnt_of_questions_per_domain:  {'reddit_eli5': 17112, 'open_qa': 1187, 'wiki_csai': 842, 'finance': 3933, 'medicine': 1248}


question_ids_all and question_ids_per_domain

In [ ]:
question_ids_all = []
question_ids_per_domain = {}
for obj in all_jsonl:
  domain = obj["source"]
  question_id = obj["question_id"]
  question_ids_all.append(question_id)
  if domain not in question_ids_per_domain:
    question_ids_per_domain[domain] = []
  question_ids_per_domain[domain].append(question_id)
# check question ids count per domain is correct
for key in question_ids_per_domain.keys():
  assert len(question_ids_per_domain[key]) == cnt_of_questions_per_domain[key]

In [ ]:
percent_per_split = {"train": 0.69, "test": 0.285, "valid": 0.025}
each_split_exact_size = {"train": 0, "test": 0, "valid": 0}
splitsize_questions_per_domain = {}
split_questions_ids_per_domain = {}
split_questions_ids_all = {"train": [], "valid":[], "test":[]}

for domain, number_of_questions in cnt_of_questions_per_domain.items():
  splitsize_questions_per_domain[domain] = {}
  
  testset_number_of_questions = int(percent_per_split['test'] * number_of_questions)
  splitsize_questions_per_domain[domain]["test"] = testset_number_of_questions

  validset_number_of_questions = int(percent_per_split['valid'] * number_of_questions)
  splitsize_questions_per_domain[domain]["valid"] = validset_number_of_questions

  trainset_number_of_questions = number_of_questions - (testset_number_of_questions + validset_number_of_questions)
  assert trainset_number_of_questions + validset_number_of_questions + testset_number_of_questions == number_of_questions
  splitsize_questions_per_domain[domain]["train"] = trainset_number_of_questions
  
  each_split_exact_size["train"] += trainset_number_of_questions
  each_split_exact_size["valid"] += validset_number_of_questions
  each_split_exact_size["test"] += testset_number_of_questions

  split_questions_ids_per_domain[domain] = {}

  split_questions_ids_per_domain[domain]["train"] = question_ids_per_domain[domain][0:trainset_number_of_questions]
  split_questions_ids_per_domain[domain]["valid"] = question_ids_per_domain[domain][trainset_number_of_questions:trainset_number_of_questions+validset_number_of_questions]
  split_questions_ids_per_domain[domain]["test"] = question_ids_per_domain[domain][trainset_number_of_questions+validset_number_of_questions:number_of_questions]
  
  split_questions_ids_all["train"] = split_questions_ids_all["train"] + split_questions_ids_per_domain[domain]["train"] 
  split_questions_ids_all["valid"] = split_questions_ids_all["valid"] + split_questions_ids_per_domain[domain]["valid"] 
  split_questions_ids_all["test"] = split_questions_ids_all["test"] + split_questions_ids_per_domain[domain]["test"] 
  
print("number of questions for train/test/valid per domain: ", splitsize_questions_per_domain)
print("number of questions for train/test/valid for all: ", each_split_exact_size)
assert (each_split_exact_size["train"] + each_split_exact_size["valid"] + each_split_exact_size["test"]) == all_questions

for domain, splits in split_questions_ids_per_domain.items():
  for split in splits:
    assert len(split_questions_ids_per_domain[domain][split]) == splitsize_questions_per_domain[domain][split]

assert len(split_questions_ids_all["train"] + split_questions_ids_all["valid"] + split_questions_ids_all["test"]) == all_questions

number of questions for train/test/valid per domain:  {'reddit_eli5': {'test': 4876, 'valid': 427, 'train': 11809}, 'open_qa': {'test': 338, 'valid': 29, 'train': 820}, 'wiki_csai': {'test': 239, 'valid': 21, 'train': 582}, 'finance': {'test': 1120, 'valid': 98, 'train': 2715}, 'medicine': {'test': 355, 'valid': 31, 'train': 862}}
number of questions for train/test/valid for all:  {'train': 16788, 'test': 6928, 'valid': 606}


re-check (besides assert) to see what is going in the data, e.g., for reddit_eli5 source, in train set, we should have 11809 questions. Let's check it and being sure:

In [ ]:
print("reddit_eli5, train size: ", len(split_questions_ids_per_domain["reddit_eli5"]["train"]))

reddit_eli5, train size:  11809


### Data split size per source (domain) and per split (train/valid/test)
##### **Number of questions for train/test/valid per domain:**  {'reddit_eli5': {'test': 4876, 'valid': 427, 'train': 11809}, 'open_qa': {'test': 338, 'valid': 29, 'train': 820}, 'wiki_csai': {'test': 239, 'valid': 21, 'train': 582}, 'finance': {'test': 1120, 'valid': 98, 'train': 2715}, 'medicine': {'test': 355, 'valid': 31, 'train': 862}}
#### **Number of questions for train/test/valid for all:**  {'train': 16788, 'test': 6928, 'valid': 606}


### qrels_h_{train/valid/test}.tsv

In [ ]:
#qrels_h_train.tsv
qrels_lines = []
qrel_line_format = "{qid}	0	{did}	1\n"
for question_id in split_questions_ids_all["train"]:
  rel_docs = qrels_all_h[question_id]
  for did in rel_docs:
    qrels_lines.append(qrel_line_format.format(qid = question_id, did = did))
output_path = base_path + "qrels_h_train.tsv"
f_w = open(output_path, "w+")
f_w.write("".join(qrels_lines))
f_w.close()
print("line cnt: ", len(qrels_lines))

line cnt:  40406


In [ ]:
#qrels_h_valid.tsv
qrels_lines = []
qrel_line_format = "{qid}	0	{did}	1\n"
for question_id in split_questions_ids_all["valid"]:
  rel_docs = qrels_all_h[question_id]
  for did in rel_docs:
    qrels_lines.append(qrel_line_format.format(qid = question_id, did = did))
output_path = base_path + "qrels_h_valid.tsv"
f_w = open(output_path, "w+")
f_w.write("".join(qrels_lines))
f_w.close()
print("line cnt: ", len(qrels_lines))

line cnt:  1460


In [ ]:
#qrels_h_test.tsv
qrels_lines = []
qrel_line_format = "{qid}	0	{did}	1\n"
for question_id in split_questions_ids_all["test"]:
  rel_docs = qrels_all_h[question_id]
  for did in rel_docs:
    qrels_lines.append(qrel_line_format.format(qid = question_id, did = did))
output_path = base_path + "qrels_h_test.tsv"
f_w = open(output_path, "w+")
f_w.write("".join(qrels_lines))
f_w.close()
print("line cnt: ", len(qrels_lines))

line cnt:  16680


### qrels_c_{train/valid/test}.tsv

In [ ]:
#qrels_c_train.tsv
qrels_lines = []
qrel_line_format = "{qid}	0	{did}	1\n"
for question_id in split_questions_ids_all["train"]:
  rel_docs = qrels_all_c[question_id]
  for did in rel_docs:
    qrels_lines.append(qrel_line_format.format(qid = question_id, did = did))
output_path = base_path + "qrels_c_train.tsv"
f_w = open(output_path, "w+")
f_w.write("".join(qrels_lines))
f_w.close()
print("line cnt: ", len(qrels_lines))

line cnt:  18452


In [ ]:
#qrels_c_valid.tsv
qrels_lines = []
qrel_line_format = "{qid}	0	{did}	1\n"
for question_id in split_questions_ids_all["valid"]:
  rel_docs = qrels_all_c[question_id]
  for did in rel_docs:
    qrels_lines.append(qrel_line_format.format(qid = question_id, did = did))
output_path = base_path + "qrels_c_valid.tsv"
f_w = open(output_path, "w+")
f_w.write("".join(qrels_lines))
f_w.close()
print("line cnt: ", len(qrels_lines))

line cnt:  672


In [ ]:
#qrels_c_test.tsv
qrels_lines = []
qrel_line_format = "{qid}	0	{did}	1\n"
for question_id in split_questions_ids_all["test"]:
  rel_docs = qrels_all_c[question_id]
  for did in rel_docs:
    qrels_lines.append(qrel_line_format.format(qid = question_id, did = did))
output_path = base_path + "qrels_c_test.tsv"
f_w = open(output_path, "w+")
f_w.write("".join(qrels_lines))
f_w.close()
print("line cnt: ", len(qrels_lines))

line cnt:  7756


### collectionandqueries.zip

In [ ]:
!zip -r ./gdrive/MyDrive/ChatGPT-RetrievalQA/collectionandqueries.zip ./gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_c_test.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_c_valid.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_c_train.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_h_test.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_h_valid.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_h_train.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/queries.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/collection_c.tsv ./gdrive/MyDrive/ChatGPT-RetrievalQA/collection_h.tsv

updating: gdrive/MyDrive/ChatGPT-RetrievalQA/collection_c.tsv (deflated 70%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/collection_h.tsv (deflated 63%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_c_test.tsv (deflated 77%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_c_train.tsv (deflated 74%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_c_valid.tsv (deflated 80%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_h_test.tsv (deflated 86%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_h_train.tsv (deflated 84%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/qrels_h_valid.tsv (deflated 85%)
updating: gdrive/MyDrive/ChatGPT-RetrievalQA/queries.tsv (deflated 65%)


### Train-H Triples (tsv: query, positive passage, negative passage)

In [ ]:
def normalize_content(content):
  return content.replace('\r', '').replace('\n', '').replace('\t', ' ')

import random
import tqdm
random.seed(42)
negative_docs_size = 1000
output_path = base_path + "train_h_triples.tsv"
output_path_qidpid = base_path + "train_h_qidpidtriples.tsv"
output_line_format = "{query}\t{positive_content}\t{neg_content}\n"
output_line_format_qidpid = "{qid}\t{positive_id}\t{neg_id}\n"
output_lines = []
all_human_answerids_list = list(all_human_answerids.keys())
# all_chatgpt_answerids = []
# all_human_answerids = []
f_w = open(output_path, "w+")
f_w_qidpid = open(output_path_qidpid, "w+")
for question_id in tqdm.tqdm(split_questions_ids_all["train"]):
  rel_docs = qrels_all_h[question_id]
  rel_docs_set = set(rel_docs) # for efficiency! check item not in rel_docs_set is O(1)
  # candidate_negs = [item for item in all_human_answerids if item not in rel_docs_set]
  # neg_ids = random.sample(candidate_negs, negative_docs_size+10) 
  neg_ids = random.sample(all_human_answerids_list, negative_docs_size+10) # 10 is added just to being sure that we will have 1k negatives. because it is possible to pos_id selected randomly and we want being able to skip that while have 1k negatives
  neg_ids_added = 0
  for pos_id in rel_docs:
    for neg_id in neg_ids:
      if neg_id == pos_id: continue
      line = output_line_format.format(
          query = normalize_content(query_dict[question_id]),
          positive_content = normalize_content(all_human_answerids[pos_id]),
          neg_content = normalize_content(all_human_answerids[neg_id]),
      )
      # output_lines.append(line)
      f_w.write(line)

      line_qidpid = output_line_format_qidpid.format(
          qid = question_id,
          positive_id = pos_id,
          neg_id = neg_id,
      )
      f_w_qidpid.write(line_qidpid)
      neg_ids_added+=1
      if neg_ids_added==negative_docs_size:
        break
      assert pos_id != neg_id
f_w.close()
f_w_qidpid.close()
print("line cnt: ", len(output_lines))

100%|██████████| 16788/16788 [13:01<00:00, 21.48it/s]

line cnt:  0


### Valid-H Triples (tsv: query, positive passage, negative passage)

In [ ]:
def normalize_content(content):
  return content.replace('\r', '').replace('\n', '').replace('\t', ' ')

import random
import tqdm
random.seed(42)
negative_docs_size = 1000
output_path = base_path + "valid_h_triples.tsv"
output_path_qidpid = base_path + "valid_h_qidpidtriples.tsv"
output_line_format = "{query}\t{positive_content}\t{neg_content}\n"
output_line_format_qidpid = "{qid}\t{positive_id}\t{neg_id}\n"
output_lines = []
all_human_answerids_list = list(all_human_answerids.keys())
# all_chatgpt_answerids = []
# all_human_answerids = []
f_w = open(output_path, "w+")
f_w_qidpid = open(output_path_qidpid, "w+")
for question_id in tqdm.tqdm(split_questions_ids_all["valid"]):
  rel_docs = qrels_all_h[question_id]
  rel_docs_set = set(rel_docs) # for efficiency! check item not in rel_docs_set is O(1)
  # candidate_negs = [item for item in all_human_answerids if item not in rel_docs_set]
  # neg_ids = random.sample(candidate_negs, negative_docs_size+10) 
  neg_ids = random.sample(all_human_answerids_list, negative_docs_size+10) # 10 is added just to being sure that we will have 1k negatives. because it is possible to pos_id selected randomly and we want being able to skip that while have 1k negatives
  neg_ids_added = 0
  for pos_id in rel_docs:
    for neg_id in neg_ids:
      if neg_id == pos_id: continue
      line = output_line_format.format(
          query = normalize_content(query_dict[question_id]),
          positive_content = normalize_content(all_human_answerids[pos_id]),
          neg_content = normalize_content(all_human_answerids[neg_id]),
      )
      # output_lines.append(line)
      f_w.write(line)

      line_qidpid = output_line_format_qidpid.format(
          qid = question_id,
          positive_id = pos_id,
          neg_id = neg_id,
      )
      f_w_qidpid.write(line_qidpid)
      neg_ids_added+=1
      if neg_ids_added==negative_docs_size:
        break
      assert pos_id != neg_id
f_w.close()
f_w_qidpid.close()
print("line cnt: ", len(output_lines))

100%|██████████| 606/606 [00:33<00:00, 17.96it/s]

line cnt:  0


### Train-C Triples (tsv: query, positive passage, negative passage)

In [ ]:
def normalize_content(content):
  return content.replace('\r', '').replace('\n', '').replace('\t', ' ')

import random
import tqdm
random.seed(42)
negative_docs_size = 1000
output_path = base_path + "train_c_triples.tsv"
output_path_qidpid = base_path + "train_c_qidpidtriples.tsv"
output_line_format = "{query}\t{positive_content}\t{neg_content}\n"
output_line_format_qidpid = "{qid}\t{positive_id}\t{neg_id}\n"
output_lines = []
# all_human_answerids_list = list(all_human_answerids.keys())
all_chatgpt_answerids_list = list(all_chatgpt_answerids.keys())
# all_chatgpt_answerids = []
# all_human_answerids = []
f_w = open(output_path, "w+")
f_w_qidpid = open(output_path_qidpid, "w+")
for question_id in tqdm.tqdm(split_questions_ids_all["train"]):
  rel_docs = qrels_all_c[question_id]
  rel_docs_set = set(rel_docs) # for efficiency! check item not in rel_docs_set is O(1)
  # candidate_negs = [item for item in all_human_answerids if item not in rel_docs_set]
  # neg_ids = random.sample(candidate_negs, negative_docs_size+10) 
  neg_ids = random.sample(all_chatgpt_answerids_list, negative_docs_size+10) # 10 is added just to being sure that we will have 1k negatives. because it is possible to pos_id selected randomly and we want being able to skip that while have 1k negatives
  neg_ids_added = 0
  for pos_id in rel_docs:
    for neg_id in neg_ids:
      if neg_id == pos_id: continue
      line = output_line_format.format(
          query = normalize_content(query_dict[question_id]),
          positive_content = normalize_content(all_chatgpt_answerids[pos_id]),
          neg_content = normalize_content(all_chatgpt_answerids[neg_id]),
      )
      # output_lines.append(line)
      f_w.write(line)

      line_qidpid = output_line_format_qidpid.format(
          qid = question_id,
          positive_id = pos_id,
          neg_id = neg_id,
      )
      f_w_qidpid.write(line_qidpid)
      neg_ids_added+=1
      if neg_ids_added==negative_docs_size:
        break
      assert pos_id != neg_id
f_w.close()
f_w_qidpid.close()
print("line cnt: ", len(output_lines))

100%|██████████| 16788/16788 [10:11<00:00, 27.44it/s]

line cnt:  0


### Valid-C Triples (tsv: query, positive passage, negative passage)

In [ ]:
def normalize_content(content):
  return content.replace('\r', '').replace('\n', '').replace('\t', ' ')

import random
import tqdm
random.seed(42)
negative_docs_size = 1000
output_path = base_path + "valid_c_triples.tsv"
output_path_qidpid = base_path + "valid_c_qidpidtriples.tsv"
output_line_format = "{query}\t{positive_content}\t{neg_content}\n"
output_line_format_qidpid = "{qid}\t{positive_id}\t{neg_id}\n"
output_lines = []
# all_human_answerids_list = list(all_human_answerids.keys())
all_chatgpt_answerids_list = list(all_chatgpt_answerids.keys())
# all_chatgpt_answerids = []
# all_human_answerids = []
f_w = open(output_path, "w+")
f_w_qidpid = open(output_path_qidpid, "w+")
for question_id in tqdm.tqdm(split_questions_ids_all["valid"]):
  rel_docs = qrels_all_c[question_id]
  rel_docs_set = set(rel_docs) # for efficiency! check item not in rel_docs_set is O(1)
  # candidate_negs = [item for item in all_human_answerids if item not in rel_docs_set]
  # neg_ids = random.sample(candidate_negs, negative_docs_size+10) 
  neg_ids = random.sample(all_chatgpt_answerids_list, negative_docs_size+10) # 10 is added just to being sure that we will have 1k negatives. because it is possible to pos_id selected randomly and we want being able to skip that while have 1k negatives
  neg_ids_added = 0
  for pos_id in rel_docs:
    for neg_id in neg_ids:
      if neg_id == pos_id: continue
      line = output_line_format.format(
          query = normalize_content(query_dict[question_id]),
          positive_content = normalize_content(all_chatgpt_answerids[pos_id]),
          neg_content = normalize_content(all_chatgpt_answerids[neg_id]),
      )
      # output_lines.append(line)
      f_w.write(line)

      line_qidpid = output_line_format_qidpid.format(
          qid = question_id,
          positive_id = pos_id,
          neg_id = neg_id,
      )
      f_w_qidpid.write(line_qidpid)
      neg_ids_added+=1
      if neg_ids_added==negative_docs_size:
        break
      assert pos_id != neg_id
f_w.close()
f_w_qidpid.close()
print("line cnt: ", len(output_lines))

100%|██████████| 606/606 [00:22<00:00, 27.47it/s]

line cnt:  0


# Re-ranking

We use Elasticsearch for indexing, and retrieving top-1000 answers per query in order to build the data for focusing on re-ranking.

## installing libraries for Elasticsearch and running Elasticsearch deamon

In [ ]:
%%capture
!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

importing libraries

In [ ]:
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm 
  import datetime
  from elasticsearch import helpers
  
except Exception as e:
  print(f"error: {e}")

Downloading elasticsearch software

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

## Starting Elasticsearch daemon instance

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

check if elasticsearch is loaded :)

In [ ]:
!curl localhost:9200

{
  "name" : "18c85f44f185",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "250L5plyQeGy_Qx07k2CnA",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


#### Elasticsearch util class

In [ ]:
import elasticsearch
from elasticsearch import helpers
print(elasticsearch.__version__)
print(elasticsearch.__versionstr__)
class ElasticSearch:
	def __init__(self):
		# configure elasticsearch
		config = {
			"localhost": 9200
		}
		self.es = elasticsearch.Elasticsearch([config, ], timeout=300)
		self.last_scroll_id = None

	def create_index(self, name, mapping, replace=False):
		if replace:
			self.delete_index(name)
		print("creating index, name: ", name)
		self.es.indices.create(index=name, body=mapping)
		print("index created successfully, index name: " + name)

	def delete_index(self, name):
		print("deleting index, name: ", name)
		self.es.indices.delete(index=name, ignore=[400, 404])
		print("index deleted successfully, index name: " + name)

	def index(self, documents, index_name, is_bulk=False):

		if is_bulk:
			try:
				# make the bulk call, and get a response
				response = helpers.bulk(self.es, documents)  # chunk_size=1000, request_timeout=200
				print("\nRESPONSE:", response)
			except Exception as e:
				print("\nERROR:", e)

	def search(self, index, body):
		try:
			# make the bulk call, and get a response
			return self.es.search(index=index, body=body)
		except Exception as e:
			print("\nERROR:", e)

	def search_all_with_scorll(self, index, body):
		try:
			# if 'size' in body:
			#     del body['size']
			#get all result ! append everything to it! append to hits[] maybe !
			# make the bulk call, and get a response'
			there_is_next_page = False

			resp = self.es.search(
				index=index,
				body=body,
				scroll = '3m', # time value for search
			)
			self.last_scroll_id = resp['_scroll_id']
			if len(resp['hits']['hits'])>=10000:
				there_is_next_page = True
			while there_is_next_page:
				resp_scroll = self.es.scroll(
					scroll = '3m', # time value for search
					scroll_id=self.last_scroll_id,
				)
				self.last_scroll_id = resp_scroll['_scroll_id']
				resp['hits']['hits'].extend(resp_scroll['hits']['hits'])
				if len(resp_scroll['hits']['hits']) >=10000:
					there_is_next_page = True
				else:
					there_is_next_page = False

			if there_is_next_page==False:
				self.last_scroll_id = None
				return resp
			#if hits is zero then there is no new!
			# return
		except Exception as e:
			print("\nERROR:", e)
	def get_with_id(self, index, id_):
		try:
			# make the bulk call, and get a response
			return self.es.get(index=index, id=id_)
		except Exception as e:
			print("\nERROR:", e)
	def termvectors(self, index, body, id):
		try:
			# make the bulk call, and get a response
			return self.es.termvectors(index=index, body=body, id=id)
		except Exception as e:
			print("\nERROR:", e)

(7, 14, 0)
7.14.0


In [ ]:
%%capture
!wget https://www.dropbox.com/s/t5t2lmm30tvhvht/paragraph-level-mapping.json

In [ ]:
def get_mapping():
  return json.loads(open("paragraph-level-mapping.json").read())

In [ ]:
def get_docs(index_name, application_in_training, docs_id_text_dict, bulk_size):
  docs = []
  for doc_id, doc_text in docs_id_text_dict.items():
    doc = {
						"_index": index_name,
						"_id": doc_id,
						"application_in_training": application_in_training,
						"content": doc_text,
      }
    docs.append(doc)
    if len(docs)>=bulk_size:
      yield docs
      docs = []
  yield docs

def indexing(index_name, application_in_training, replace_index, mapping, docs_id_text_dict, bulk_size = 500):
  print("indexer run..  index_name: {}", index_name)

  es = ElasticSearch()
  print("creating index mapping...")
  try:
    es.create_index(index_name, mapping, replace=replace_index)
  except Exception as e:
    print("creating index error: ", e)
    pass
  print("index mapping created !")
  docs = get_docs(index_name, application_in_training, docs_id_text_dict, bulk_size)
  for bulk_docs in docs:
    print("indexing {} documents started...: ".format(bulk_size))
    es.index(index_name=index_name, documents=bulk_docs, is_bulk=True)
    print("all {} docs indexed :)".format(bulk_size))
    docs = []

#### rank function

In [ ]:
def	rank(index_name, query_dict, qids, top_n, output_path):
  report = {}
  print("index_name: {}, query_dict: {}, qids: {}, output_path: {}".format(index_name, query_dict, qids, output_path))
  es = ElasticSearch()
  field = "content"
  f_w = open(output_path, "w")
  cnt = 0
  for query_id in qids:
    report[query_id] = {"count_of_retrieved_docs": 0}
    query_text  = query_dict[query_id]
    cnt+=1
    bool_query = {
        "size": top_n,
        "query": {
            "bool": {
                "should": [
                    {"match": {'content': query_text.lower()}}
                ],
                "must_not": [
                    {"term": {'_id': query_id}}
                ]
                , "minimum_should_match": 0,
                "boost": 1.0
            }
        },
			"_source": [
				"_id",
			],
    }
    candidates = es.search(index=index_name, body=bool_query)
    rank = 1
    trec_out = ""
    report[query_id]["count_of_retrieved_docs"] =  len(candidates['hits']['hits'])
    for candidate in candidates['hits']['hits']:
      document_id = candidate["_id"]
      score = candidate["_score"]
      # candidate_doc = candidate["_source"]
      line = "{query_id} Q0 {document_id} {rank} {score} STANDARD\n".format(query_id=query_id,
          document_id=document_id,
          rank=rank,
          score=score
      )
      # trec_out += line
      f_w.write(line)
      rank += 1
  f_w.close()
  es.es.transport.close()
  return report

## indexing collection-h 

In [ ]:
index_name = "collection_h"
docs_id_text_dict = dict(all_human_answerids)
application_in_training = "corpus"
replace_index = True #if index already exist, remove it first
mapping = get_mapping()
indexing(index_name, application_in_training, replace_index, mapping, docs_id_text_dict, bulk_size = 20000)

indexer run..  index_name: {} collection_h
creating index mapping...
deleting index, name:  collection_h
index deleted successfully, index name: collection_h
creating index, name:  collection_h
index created successfully, index name: collection_h
index mapping created !
indexing 20000 documents started...: 

RESPONSE: (20000, [])
all 20000 docs indexed :)
indexing 20000 documents started...: 

RESPONSE: (20000, [])
all 20000 docs indexed :)
indexing 20000 documents started...: 

RESPONSE: (18546, [])
all 20000 docs indexed :)


check the size of index now!

In [ ]:
!curl localhost:9200/_cat/indices?v

health status index        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   collection_h IT8JJ1IMRU-jYkIAx9Gq3A   1   1      55500            0     54.3mb         54.3mb
yellow open   collection_c 7i14TubDRva6i_1gThJRUg   1   1      26884            0       50mb           50mb


## indexing collection-c 

In [ ]:
index_name = "collection_c"
docs_id_text_dict = dict(all_chatgpt_answerids)
application_in_training = "corpus"
replace_index = True #if index already exist, remove it first
mapping = get_mapping()
indexing(index_name, application_in_training, replace_index, mapping, docs_id_text_dict, bulk_size = 20000)

indexer run..  index_name: {} collection_c
creating index mapping...
deleting index, name:  collection_c
index deleted successfully, index name: collection_c
creating index, name:  collection_c
index created successfully, index name: collection_c
index mapping created !
indexing 20000 documents started...: 

RESPONSE: (20000, [])
all 20000 docs indexed :)
indexing 20000 documents started...: 

RESPONSE: (6882, [])
all 20000 docs indexed :)


In [ ]:
!curl localhost:9200/_cat/indices?v

health status index        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   collection_c VJqfPmFMReKFBuzR47s_XA   1   1      24387            0     48.6mb         48.6mb
yellow open   collection_h IT8JJ1IMRU-jYkIAx9Gq3A   1   1      58546            0     82.8mb         82.8mb


## top-h (train/validation/test)

### top-1000-h train

In [ ]:
# uncomment all of the below lines to generate the file! 
index_name = "collection_h"
top_n = 1000
split_name = "train"
output_path = base_path + "top_{}_h_{}.run".format(top_n, split_name)
report_dict = rank(index_name, query_dict, split_questions_ids_all[split_name], top_n, output_path)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Check the report and run file. It is important to the output of "# of queries with 10 candidates:" be zero, so for each query, at least 10 candidate exsit!

In [ ]:
print("-----\nstat on this run: ")
print("# of queries: ", len(report_dict.keys()))
count_of_retrieved_docs_list = [report_dict[item]["count_of_retrieved_docs"] for item in report_dict.keys()]
print("# of queries with 0 candidate: ", len([item for item in count_of_retrieved_docs_list if item == 0]))
print("# of queries with >=10 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 10]))
print("# of queries with >=100 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with >=500 candidate: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with <1000 candidate: ", len([item for item in count_of_retrieved_docs_list if item < 1000]))
average_of_candidate_doc_per_query = sum(count_of_retrieved_docs_list)/len(count_of_retrieved_docs_list)
print("average_of_candidate_doc_per_query: ", average_of_candidate_doc_per_query)
print("------------------")
#take a look on file: 
!tail -n 5 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_train.run
print("------------------")
#Check count of lines. (should be <= 16788 \* 1000 = 16,788,000)
!wc -l ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_train.run

-----
stat on this run: 
# of queries:  16788
# of queries with 0 candidate:  0
# of queries with >=10 candidates:  16788
# of queries with >=100 candidates:  16783
# of queries with >=500 candidate:  16783
# of queries with <1000 candidate:  27
average_of_candidate_doc_per_query:  999.1733380986419
------------------
23935 Q0 5971_H_1 996 26.517561 STANDARD
23935 Q0 10984_H_1 997 26.506844 STANDARD
23935 Q0 13987_H_1 998 26.506844 STANDARD
23935 Q0 12347_H_2 999 26.506145 STANDARD
23935 Q0 11204_H_0 1000 26.5013 STANDARD
------------------
16774122 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_train.run


### top-1000-h valid

In [ ]:
# uncomment all of the below lines to generate the file! 
index_name = "collection_h"
top_n = 1000
split_name = "valid"
output_path = base_path + "top_{}_h_{}.run".format(top_n, split_name)
report_dict = rank(index_name, query_dict, split_questions_ids_all[split_name], top_n, output_path)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Check the report and run file. It is important to the output of "# of queries with 10 candidates:" be zero, so for each query, at least 10 candidate exsit!

In [ ]:
print("-----\nstat on this run: ")
print("# of queries: ", len(report_dict.keys()))
count_of_retrieved_docs_list = [report_dict[item]["count_of_retrieved_docs"] for item in report_dict.keys()]
print("# of queries with 0 candidate: ", len([item for item in count_of_retrieved_docs_list if item == 0]))
print("# of queries with >=10 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 10]))
print("# of queries with >=100 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with >=500 candidate: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with <1000 candidate: ", len([item for item in count_of_retrieved_docs_list if item < 1000]))
average_of_candidate_doc_per_query = sum(count_of_retrieved_docs_list)/len(count_of_retrieved_docs_list)
print("average_of_candidate_doc_per_query: ", average_of_candidate_doc_per_query)
print("------------------")
#take a look on file: 
!tail -n 5 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_valid.run
print("------------------")
#Check count of lines. 
!wc -l ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_valid.run

-----
stat on this run: 
# of queries:  606
# of queries with 0 candidate:  0
# of queries with >=10 candidates:  606
# of queries with >=100 candidates:  606
# of queries with >=500 candidate:  606
# of queries with <1000 candidate:  1
average_of_candidate_doc_per_query:  999.927392739274
------------------
23966 Q0 4904_H_2 996 28.89788 STANDARD
23966 Q0 1481_H_0 997 28.896053 STANDARD
23966 Q0 223_H_1 998 28.892103 STANDARD
23966 Q0 9602_H_1 999 28.885115 STANDARD
23966 Q0 7642_H_0 1000 28.884119 STANDARD
------------------
605956 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_valid.run


### top-1000-h test

In [ ]:
# uncomment all of the below lines to generate the file! 
index_name = "collection_h"
top_n = 1000
split_name = "test"
output_path = base_path + "top_{}_h_{}.run".format(top_n, split_name)
report_dict = rank(index_name, query_dict, split_questions_ids_all[split_name], top_n, output_path)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Check the report and run file. It is important to the output of "# of queries with 10 candidates:" be zero, so for each query, at least 10 candidate exsit!

In [ ]:
print("-----\nstat on this run: ")
print("# of queries: ", len(report_dict.keys()))
count_of_retrieved_docs_list = [report_dict[item]["count_of_retrieved_docs"] for item in report_dict.keys()]
print("# of queries with 0 candidate: ", len([item for item in count_of_retrieved_docs_list if item == 0]))
print("# of queries with >=10 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 10]))
print("# of queries with >=100 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with >=500 candidate: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with <1000 candidate: ", len([item for item in count_of_retrieved_docs_list if item < 1000]))
average_of_candidate_doc_per_query = sum(count_of_retrieved_docs_list)/len(count_of_retrieved_docs_list)
print("average_of_candidate_doc_per_query: ", average_of_candidate_doc_per_query)
print("------------------")
#take a look on file: 
!tail -n 5 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_test.run
print("------------------")
#Check count of lines. 
!wc -l ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_test.run

-----
stat on this run: 
# of queries:  6928
# of queries with 0 candidate:  0
# of queries with >=10 candidates:  6928
# of queries with >=100 candidates:  6925
# of queries with >=500 candidate:  6925
# of queries with <1000 candidate:  12
average_of_candidate_doc_per_query:  998.9672344110854
------------------
24321 Q0 4786_H_2 996 35.236233 STANDARD
24321 Q0 3624_H_1 997 35.235703 STANDARD
24321 Q0 6051_H_0 998 35.234554 STANDARD
24321 Q0 21125_H_0 999 35.229797 STANDARD
24321 Q0 8982_H_1 1000 35.2249 STANDARD
------------------
6920845 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_h_test.run


## top-c (train/validation/test)

### top-1000-c train

In [ ]:
# uncomment all of the below lines to generate the file! 
index_name = "collection_c"
top_n = 1000
split_name = "train"
output_path = base_path + "top_{}_c_{}.run".format(top_n, split_name)
report_dict = rank(index_name, query_dict, split_questions_ids_all[split_name], top_n, output_path)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Check the report and run file. It is important to the output of "# of queries with 10 candidates:" be zero, so for each query, at least 10 candidate exsit!

In [ ]:
print("-----\nstat on this run: ")
print("# of queries: ", len(report_dict.keys()))
count_of_retrieved_docs_list = [report_dict[item]["count_of_retrieved_docs"] for item in report_dict.keys()]
print("# of queries with 0 candidate: ", len([item for item in count_of_retrieved_docs_list if item == 0]))
print("# of queries with >=10 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 10]))
print("# of queries with >=100 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with >=500 candidate: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with <1000 candidate: ", len([item for item in count_of_retrieved_docs_list if item < 1000]))
average_of_candidate_doc_per_query = sum(count_of_retrieved_docs_list)/len(count_of_retrieved_docs_list)
print("average_of_candidate_doc_per_query: ", average_of_candidate_doc_per_query)
print("------------------")
#take a look on file: 
!tail -n 5 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_train.run
print("------------------")
#Check count of lines. (should be <= 16788 \* 1000 = 16,788,000)
!wc -l ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_train.run

-----
stat on this run: 
# of queries:  16788
# of queries with 0 candidate:  0
# of queries with >=10 candidates:  16788
# of queries with >=100 candidates:  16784
# of queries with >=500 candidate:  16784
# of queries with <1000 candidate:  35
average_of_candidate_doc_per_query:  998.8104598522755
------------------
23935 Q0 1070_C_0 996 17.58851 STANDARD
23935 Q0 6746_C_0 997 17.587818 STANDARD
23935 Q0 4025_C_0 998 17.583414 STANDARD
23935 Q0 22790_C_0 999 17.580814 STANDARD
23935 Q0 2713_C_0 1000 17.578367 STANDARD
------------------
16768030 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_train.run


### top-1000-c valid

In [ ]:
# uncomment all of the below lines to generate the file! 
index_name = "collection_c"
top_n = 1000
split_name = "valid"
output_path = base_path + "top_{}_c_{}.run".format(top_n, split_name)
report_dict = rank(index_name, query_dict, split_questions_ids_all[split_name], top_n, output_path)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Check the report and run file. It is important to the output of "# of queries with 10 candidates:" be zero, so for each query, at least 10 candidate exsit!

In [ ]:
print("-----\nstat on this run: ")
print("# of queries: ", len(report_dict.keys()))
count_of_retrieved_docs_list = [report_dict[item]["count_of_retrieved_docs"] for item in report_dict.keys()]
print("# of queries with 0 candidate: ", len([item for item in count_of_retrieved_docs_list if item == 0]))
print("# of queries with >=10 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 10]))
print("# of queries with >=100 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with >=500 candidate: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with <1000 candidate: ", len([item for item in count_of_retrieved_docs_list if item < 1000]))
average_of_candidate_doc_per_query = sum(count_of_retrieved_docs_list)/len(count_of_retrieved_docs_list)
print("average_of_candidate_doc_per_query: ", average_of_candidate_doc_per_query)
print("------------------")
#take a look on file: 
!tail -n 5 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_valid.run
print("------------------")
#Check count of lines. 
!wc -l ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_valid.run

-----
stat on this run: 
# of queries:  606
# of queries with 0 candidate:  0
# of queries with >=10 candidates:  606
# of queries with >=100 candidates:  606
# of queries with >=500 candidate:  606
# of queries with <1000 candidate:  2
average_of_candidate_doc_per_query:  999.6584158415842
------------------
23966 Q0 9400_C_0 996 19.34834 STANDARD
23966 Q0 21618_C_0 997 19.347855 STANDARD
23966 Q0 13383_C_0 998 19.347305 STANDARD
23966 Q0 17374_C_2 999 19.342844 STANDARD
23966 Q0 15085_C_0 1000 19.341434 STANDARD
------------------
605793 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_valid.run


### top-1000-c test

In [ ]:
# uncomment all of the below lines to generate the file! 
index_name = "collection_c"
top_n = 1000
split_name = "test"
output_path = base_path + "top_{}_c_{}.run".format(top_n, split_name)
report_dict = rank(index_name, query_dict, split_questions_ids_all[split_name], top_n, output_path)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Check the report and run file. It is important to the output of "# of queries with 10 candidates:" be zero, so for each query, at least 10 candidate exsit!

In [ ]:
print("-----\nstat on this run: ")
print("# of queries: ", len(report_dict.keys()))
count_of_retrieved_docs_list = [report_dict[item]["count_of_retrieved_docs"] for item in report_dict.keys()]
print("# of queries with 0 candidate: ", len([item for item in count_of_retrieved_docs_list if item == 0]))
print("# of queries with >=10 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 10]))
print("# of queries with >=100 candidates: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with >=500 candidate: ", len([item for item in count_of_retrieved_docs_list if item >= 100]))
print("# of queries with <1000 candidate: ", len([item for item in count_of_retrieved_docs_list if item < 1000]))
average_of_candidate_doc_per_query = sum(count_of_retrieved_docs_list)/len(count_of_retrieved_docs_list)
print("average_of_candidate_doc_per_query: ", average_of_candidate_doc_per_query)
print("------------------")
#take a look on file: 
!tail -n 5 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_test.run
print("------------------")
#Check count of lines. 
!wc -l ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_test.run

-----
stat on this run: 
# of queries:  6928
# of queries with 0 candidate:  0
# of queries with >=10 candidates:  6928
# of queries with >=100 candidates:  6924
# of queries with >=500 candidate:  6924
# of queries with <1000 candidate:  18
average_of_candidate_doc_per_query:  998.5010103926097
------------------
24321 Q0 8195_C_0 996 22.96967 STANDARD
24321 Q0 8704_C_0 997 22.96967 STANDARD
24321 Q0 9207_C_0 998 22.969248 STANDARD
24321 Q0 19730_C_0 999 22.963724 STANDARD
24321 Q0 16679_C_0 1000 22.961594 STANDARD
------------------
6917615 ./gdrive/MyDrive/ChatGPT-RetrievalQA/top_1000_c_test.run
